In [1]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///./User_Information.db', echo=True)

In [2]:
from sqlalchemy.orm import declarative_base

Base = declarative_base()

In [3]:
from sqlalchemy import Column, Integer, String


class User(Base):
    __tablename__ = 'deepmass_user'
    id = Column(Integer, primary_key=True, unique=True)
    name = Column(String)
    contact_info = Column(String, unique=True)
    passwd = Column(String)

    def _repr_(self):
        return "<User(name='%s', contact_info='%s', passwd='%s')>" % (
            self.name, self.contact_info, self.passwd
        )

In [4]:
User.__table__

Table('users', MetaData(), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('contact_info', String(), table=<users>), Column('passwd', String(), table=<users>), schema=None)

In [5]:
Base.metadata.create_all(engine)

2023-12-26 09:42:55,491 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-26 09:42:55,492 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2023-12-26 09:42:55,493 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-12-26 09:42:55,494 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2023-12-26 09:42:55,495 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-12-26 09:42:55,498 INFO sqlalchemy.engine.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	contact_info VARCHAR, 
	passwd VARCHAR, 
	PRIMARY KEY (id), 
	UNIQUE (id), 
	UNIQUE (contact_info)
)


2023-12-26 09:42:55,499 INFO sqlalchemy.engine.Engine [no key 0.00071s] ()
2023-12-26 09:42:55,505 INFO sqlalchemy.engine.Engine COMMIT


In [6]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

In [7]:
ed_user = User(name='Lysa', contact_info='Lysa_Jones@outlook.com', passwd='Lysa_Jones0000')
session.add(ed_user)
ed_user.name

'Lysa'

In [8]:
ed_user = User(name='Nana', contact_info='Lnanalo@outlook.com', passwd='nana0000111')
session.add(ed_user)
session.commit()

2023-12-26 09:43:10,563 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-26 09:43:10,566 INFO sqlalchemy.engine.Engine INSERT INTO users (name, contact_info, passwd) VALUES (?, ?, ?) RETURNING id
2023-12-26 09:43:10,567 INFO sqlalchemy.engine.Engine [generated in 0.00013s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('Lysa', 'Lysa_Jones@outlook.com', 'Lysa_Jones0000')
2023-12-26 09:43:10,569 INFO sqlalchemy.engine.Engine INSERT INTO users (name, contact_info, passwd) VALUES (?, ?, ?) RETURNING id
2023-12-26 09:43:10,570 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('Nana', 'Lnanalo@outlook.com', 'nana0000111')
2023-12-26 09:43:10,571 INFO sqlalchemy.engine.Engine COMMIT


In [26]:
our_user = session.query(User).filter_by(name='ed').first()
our_user


2023-12-25 17:08:54,179 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.contact_info AS users_contact_info, users.passwd AS users_passwd 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2023-12-25 17:08:54,181 INFO sqlalchemy.engine.Engine [generated in 0.00140s] ('ed', 1, 0)


In [37]:
session.rollback()

2023-12-25 17:10:18,684 INFO sqlalchemy.engine.Engine ROLLBACK


In [40]:
ed_user.contact_info

'Lnanalo@outlook.com'

In [55]:
for instance in session.query(User).order_by(User.id):
    print(instance.name, instance.contact_info, instance.passwd)



2023-12-25 17:22:02,493 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-25 17:22:02,494 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.contact_info AS users_contact_info, users.passwd AS users_passwd 
FROM users ORDER BY users.id
2023-12-25 17:22:02,495 INFO sqlalchemy.engine.Engine [generated in 0.00108s] ()
Lysa Lysa_Jones@outlook.com Lysa_Jones0000
Nana Lnanalo@outlook.com nana0000111


In [71]:



def login(name, passwd):
    results = session.execute(select(User).where(User.name.in_([name])).where(User.passwd.in_([passwd])))
    res = results.scalars().all()

    if len(res) > 0:
        return True
    else:
        return False


login("Nana", 'nana0000111')

2023-12-27 09:41:35,023 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-27 09:41:35,029 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.contact_info, users.passwd 
FROM users 
WHERE users.name IN (?) AND users.passwd IN (?)
2023-12-27 09:41:35,031 INFO sqlalchemy.engine.Engine [cached since 8.596e+04s ago] ('Nana', 'nana0000111')


True

In [72]:

from sqlalchemy import select

In [73]:
session.rollback()

2023-12-27 09:41:40,749 INFO sqlalchemy.engine.Engine ROLLBACK


In [78]:


def register(name, email, passwd):
    existing_user = session.query(User).filter_by(contact_info=email).first()

    if existing_user:
        print('该用户已存在，可直接登录或重新输入用户名进行注册')
    else:
        new_user = User(name=name, contact_info=email, passwd=passwd)
        session.add(new_user)
        session.commit()
        print('注册成功')


register('Dolya', 'Dolya@qq.com', '1234560')


2023-12-28 09:47:38,218 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-28 09:47:38,219 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.contact_info AS users_contact_info, users.passwd AS users_passwd 
FROM users 
WHERE users.contact_info = ?
 LIMIT ? OFFSET ?
2023-12-28 09:47:38,220 INFO sqlalchemy.engine.Engine [cached since 1.555e+05s ago] ('Dolya@qq.com', 1, 0)
该用户已存在，可直接登录或重新输入用户名进行注册


In [50]:
import hashlib

m = hashlib.md5()
m.update(b'123')
m.hexdigest()

'202cb962ac59075b964b07152d234b70'

In [51]:
hashlib.new('md5', b'123').hexdigest()

'202cb962ac59075b964b07152d234b70'

In [83]:
def encrypt(passwd):
    if not passwd.encode('utf-8').isascii():
        return "输入错误,密码长度至少8个字符,包含数字,大小写字母和特殊字符其中两种"
    new_passwd = hashlib.new('md5', passwd.encode('utf-8')).hexdigest()
    return new_passwd


sd = encrypt('krL123')
print(sd)

f5c666d8258941c6f59098a07a0b129c


In [69]:
import uuid

uuid.uuid1().hex

'5abdec36a3c711ee87fd00d861a39b83'

In [3]:
from matchms.importing import load_from_mgf

sp_list = list(load_from_mgf('./data/CYY_P_MS2.mgf'))

In [4]:
sp_list[0]

In [7]:
sp_list[0].metadata

{'feature_id': '74',
 'pepmass': (100.07616, None),
 'scans': '74',
 'charge': 1,
 'mslevel': '2',
 'retention_time': 174.3,
 'precursor_mz': 100.07616}

In [9]:
sp_list[0].__dict__

{'_metadata': <matchms.Metadata.Metadata at 0x2446aadf7c0>,
 '_peaks': <matchms.Fragments.Fragments at 0x2446aadfbb0>,
 '_losses': None}

In [12]:
list(load_from_mgf('./example/minimum_example.mgf'))[0].metadata

{'charge': -1,
 'precursor_mz': 194.04588176800002,
 'compound_name': 'challenge_255',
 'challenge': 'CASMI2016_Challenge-006',
 'formula': 'C9H9NO4',
 'smiles': 'CC(=O)NC1=CC=C(O)C(=C1)C(O)=O',
 'inchikey': 'GEFDRROBUCULOD-UHFFFAOYSA-N',
 'adduct': '[M-H]-',
 'ionmode': 'negative',
 'parent_mass': '195.053157768'}